# Python处理Word文档详解 - 自动化与批量处理

## 📚 学习目标
本教程重点讲解Python处理Word文档的**独有优势**，而非简单的文档编辑功能。

### Python处理Word的核心优势：
1. **批量自动化处理** - 一次性处理成百上千个Word文档
2. **数据提取与整合** - 从多个Word中提取数据并汇总分析
3. **模板化批量生成** - 根据数据批量生成个性化文档
4. **自动化格式处理** - 批量统一格式、样式
5. **与数据源集成** - 从数据库、Excel等生成Word报告

### 主要学习的库：
- **python-docx** - Word文档读写和操作
- **docxtpl** - Word模板引擎（支持Jinja2语法）
- **pandas** - 数据处理（配合使用）

### 与Word软件对比：
- ❌ Word软件：手动打开、编辑、保存（每个文档5-10分钟）
- ✅ Python：编写脚本，批量处理1000个文档（几分钟完成）

## 📦 安装必要的库

In [1]:
# 安装Word处理相关的库
import sys
import subprocess

libraries = [
    'python-docx',  # Word文档处理核心库
    'docxtpl',      # Word模板引擎
    'pandas',       # 数据处理
]

print("正在检查并安装必要的库...\\n")
for lib in libraries:
    try:
        if lib == 'python-docx':
            __import__('docx')
        else:
            __import__(lib)
        print(f"✓ {lib} 已安装")
    except ImportError:
        print(f"⏳ 正在安装 {lib}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", lib, "-q"])
        print(f"✓ {lib} 安装完成")

print("\\n所有库已准备就绪！")

正在检查并安装必要的库...\n
⏳ 正在安装 python-docx...
✓ python-docx 安装完成
⏳ 正在安装 docxtpl...
✓ docxtpl 安装完成
⏳ 正在安装 pandas...
✓ pandas 安装完成
\n所有库已准备就绪！


## 导入所需库

In [2]:
# 导入Word处理库
from docx import Document
from docx.shared import Pt, Inches, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.style import WD_STYLE_TYPE
from docxtpl import DocxTemplate

# 导入辅助库
import pandas as pd
from pathlib import Path
import os
from datetime import datetime

print("✓ 所有库导入成功！")
print(f"python-docx 已加载")

✓ 所有库导入成功！
python-docx 已加载


---
# 第一部分：批量创建Word文档

## ⭐ Python独特优势：程序化批量生成文档

### 应用场景：
- 批量生成合同、协议（如为100个客户生成合同）
- 批量生成报告、证书
- 根据数据自动生成文档
- 定时自动生成周报、月报

### Python vs Word：
- Word：打开模板 → 手动填写 → 另存为（重复100次）
- Python：运行脚本，自动生成100份文档

### 1. 创建测试目录

In [3]:
# 创建测试目录
test_dir = Path("word_test_files")
test_dir.mkdir(exist_ok=True)

print(f"✓ 测试目录已创建: {test_dir.absolute()}")

✓ 测试目录已创建: c:\Users\Administrator\Desktop\cursor_python_learn\word_test_files


### ⭐ 重点复习：创建基础Word文档

**python-docx的核心功能：**
- 程序化创建文档内容（标题、段落、表格）
- 设置样式和格式
- 完全自动化，无需手动操作

**实际应用：**
- 自动生成日报、周报
- 根据数据生成报告
- 批量创建标准格式文档

In [4]:
# 创建一个简单的Word文档
def create_simple_document(filename):
    """
    创建一个包含标题、段落、表格的Word文档
    """
    # 创建文档对象
    doc = Document()
    
    # 添加标题
    doc.add_heading('销售月度报告', 0)
    
    # 添加段落
    doc.add_paragraph(f'报告日期: {datetime.now().strftime("%Y-%m-%d")}')
    doc.add_paragraph('本月销售情况总结如下：')
    
    # 添加二级标题
    doc.add_heading('销售数据', 2)
    
    # 添加表格
    table = doc.add_table(rows=4, cols=3)
    table.style = 'Light Grid Accent 1'
    
    # 填充表头
    header_cells = table.rows[0].cells
    header_cells[0].text = '产品'
    header_cells[1].text = '销量'
    header_cells[2].text = '销售额'
    
    # 填充数据
    data = [
        ('产品A', '1200', '¥60,000'),
        ('产品B', '850', '¥42,500'),
        ('产品C', '2000', '¥80,000'),
    ]
    
    for i, (product, quantity, amount) in enumerate(data, 1):
        cells = table.rows[i].cells
        cells[0].text = product
        cells[1].text = quantity
        cells[2].text = amount
    
    # 添加总结段落
    doc.add_paragraph()
    doc.add_paragraph('本月总销售额达到 ¥182,500，完成目标的 91.2%。')
    
    # 保存文档
    doc.save(filename)
    print(f"✓ 文档已创建: {filename}")

# 创建示例文档
create_simple_document(test_dir / "sales_report_example.docx")

✓ 文档已创建: word_test_files\sales_report_example.docx


### ⭐ 重点复习：批量生成个性化文档

**Python的杀手级应用！**

**应用场景：**
- 为每个员工生成工资单
- 为每个客户生成合同
- 批量生成证书、邀请函

**效率对比：**
- 手动：每份文档5分钟，100份需要8小时
- Python：运行脚本，100份只需1分钟

In [5]:
# 批量生成个性化文档
def batch_create_personalized_docs(data_list, template_func, output_dir):
    """
    批量生成个性化文档
    
    参数：
        data_list: 数据列表（每个元素对应一个文档）
        template_func: 生成文档的模板函数
        output_dir: 输出目录
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)
    
    print(f"开始批量生成 {len(data_list)} 份文档...\\n")
    
    for i, data in enumerate(data_list, 1):
        try:
            filename = output_dir / f"{data['name']}_工资单.docx"
            template_func(data, filename)
            print(f"[{i}/{len(data_list)}] ✓ {filename.name}")
        except Exception as e:
            print(f"[{i}/{len(data_list)}] ✗ 生成失败: {e}")
    
    print(f"\\n✓ 批量生成完成！")

# 定义工资单模板函数
def create_salary_slip(employee_data, filename):
    """
    生成员工工资单
    """
    doc = Document()
    
    # 标题
    title = doc.add_heading('工资单', 0)
    title.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    # 基本信息
    doc.add_paragraph(f"姓名：{employee_data['name']}")
    doc.add_paragraph(f"部门：{employee_data['department']}")
    doc.add_paragraph(f"月份：{employee_data['month']}")
    doc.add_paragraph()
    
    # 工资明细表格
    table = doc.add_table(rows=6, cols=2)
    table.style = 'Light List Accent 1'
    
    # 填充数据
    items = [
        ('基本工资', f"¥{employee_data['base_salary']:,}"),
        ('绩效奖金', f"¥{employee_data['bonus']:,}"),
        ('补贴', f"¥{employee_data['allowance']:,}"),
        ('扣除项', f"¥{employee_data['deduction']:,}"),
        ('',  ''),
        ('实发工资', f"¥{employee_data['net_salary']:,}"),
    ]
    
    for i, (item, value) in enumerate(items):
        cells = table.rows[i].cells
        cells[0].text = item
        cells[1].text = value
    
    doc.save(filename)

# 示例数据（模拟从数据库或Excel读取）
employees = [
    {'name': '张三', 'department': '销售部', 'month': '2024-01', 
     'base_salary': 8000, 'bonus': 2000, 'allowance': 500, 'deduction': 800, 'net_salary': 9700},
    {'name': '李四', 'department': '技术部', 'month': '2024-01',
     'base_salary': 12000, 'bonus': 3000, 'allowance': 1000, 'deduction': 1200, 'net_salary': 14800},
    {'name': '王五', 'department': '市场部', 'month': '2024-01',
     'base_salary': 9000, 'bonus': 1500, 'allowance': 600, 'deduction': 900, 'net_salary': 10200},
]

# 批量生成工资单
batch_create_personalized_docs(employees, create_salary_slip, test_dir / "salary_slips")

print("\\n💡 实际应用：")
print("可以从数据库或Excel读取员工数据：")
print("df = pd.read_excel('employees.xlsx')")
print("batch_create_personalized_docs(df.to_dict('records'), ...)")

开始批量生成 3 份文档...\n
[1/3] ✓ 张三_工资单.docx
[2/3] ✓ 李四_工资单.docx
[3/3] ✓ 王五_工资单.docx
\n✓ 批量生成完成！
\n💡 实际应用：
可以从数据库或Excel读取员工数据：
df = pd.read_excel('employees.xlsx')
batch_create_personalized_docs(df.to_dict('records'), ...)


---
# 第二部分：从Word提取数据

## ⭐ Python独特优势：批量数据提取与汇总

### 应用场景：
- 从100份合同中提取关键信息
- 从所有报告中提取数据并汇总
- 提取Word表格数据进行分析
- 批量提取文档元数据

### Python vs 手动：
- 手动：打开每个文档 → 复制数据 → 粘贴到Excel（几天）
- Python：运行脚本，自动提取并汇总（几分钟）

### ⭐ 重点复习：提取Word中的文本内容

**应用场景：**
- 批量提取文档摘要
- 搜索包含特定关键词的文档
- 提取所有文档的正文进行分析

In [ ]:
# 从Word文档提取文本
def extract_text_from_docx(docx_path):
    """
    提取Word文档中的所有文本
    
    返回：文本内容
    """
    doc = Document(docx_path)
    
    print(f"正在提取: {docx_path.name}")
    print(f"段落数: {len(doc.paragraphs)}\\n")
    
    # 提取所有段落文本
    full_text = []
    for para in doc.paragraphs:
        if para.text.strip():  # 忽略空段落
            full_text.append(para.text)
    
    return '\\n'.join(full_text)

# 示例：提取文档内容
text = extract_text_from_docx(test_dir / "sales_report_example.docx")
print("提取的文本内容：")
print(text[:200] + "...")  # 显示前200个字符

### ⭐ 重点复习：提取Word表格数据

**Python的核心优势：**
- 自动识别并提取所有表格
- 转换为DataFrame进行数据分析
- 批量提取多个文档的表格并汇总

**实际应用：**
- 从所有季度报告中提取财务表格
- 汇总多个文档中的数据表
- 提取表格数据导入数据库

In [ ]:
# 从Word文档提取表格数据
def extract_tables_from_docx(docx_path):
    """
    提取Word文档中的所有表格
    
    返回：表格数据列表（每个表格转换为DataFrame）
    """
    doc = Document(docx_path)
    tables_data = []
    
    print(f"正在提取表格: {docx_path.name}")
    print(f"找到 {len(doc.tables)} 个表格\\n")
    
    for table_num, table in enumerate(doc.tables, 1):
        # 提取表格数据
        data = []
        for row in table.rows:
            row_data = [cell.text.strip() for cell in row.cells]
            data.append(row_data)
        
        # 转换为DataFrame（假设第一行是表头）
        if len(data) > 1:
            df = pd.DataFrame(data[1:], columns=data[0])
            tables_data.append({
                'table_num': table_num,
                'data': df
            })
            
            print(f"表格 {table_num}:")
            print(df.to_string(index=False))
            print()
    
    return tables_data

# 提取表格数据
tables = extract_tables_from_docx(test_dir / "sales_report_example.docx")

# 演示数据分析
if tables:
    df = tables[0]['data']
    print("📊 数据分析示例：")
    print(f"产品种类: {len(df)}")
    print(f"\\n可以继续进行数据清洗和分析...")

### ⭐ 重点复习：批量提取并汇总数据

**这是Python处理Word的杀手级功能！**

**应用场景：**
- 从100份报告中提取关键数据并生成汇总表
- 从所有合同中提取金额信息
- 批量提取元数据（创建日期、修改日期等）

In [ ]:
# 批量提取多个Word文档的数据并汇总
def batch_extract_and_summarize(docx_directory, file_pattern="*.docx"):
    """
    批量提取多个Word文档的数据并汇总
    
    返回：汇总数据的DataFrame
    """
    all_data = []
    docx_files = list(Path(docx_directory).glob(file_pattern))
    
    print(f"找到 {len(docx_files)} 个Word文档\\n")
    
    for docx_file in docx_files:
        try:
            doc = Document(docx_file)
            
            # 提取文档信息
            doc_info = {
                '文件名': docx_file.name,
                '段落数': len(doc.paragraphs),
                '表格数': len(doc.tables),
                '字符数': sum(len(p.text) for p in doc.paragraphs),
                '文件大小(KB)': docx_file.stat().st_size / 1024,
            }
            
            # 提取文档核心属性（如果有）
            if hasattr(doc, 'core_properties'):
                props = doc.core_properties
                doc_info['标题'] = props.title or 'N/A'
                doc_info['作者'] = props.author or 'N/A'
            
            all_data.append(doc_info)
            print(f"✓ {docx_file.name}")
        
        except Exception as e:
            print(f"✗ {docx_file.name}: {e}")
    
    # 转换为DataFrame
    df = pd.DataFrame(all_data)
    
    print("\\n" + "="*60)
    print("📊 批量提取汇总报告：\\n")
    print(df.to_string(index=False))
    
    print("\\n统计摘要：")
    print(f"- 总文件数: {len(df)}")
    print(f"- 总段落数: {df['段落数'].sum()}")
    print(f"- 总字符数: {df['字符数'].sum():,}")
    print(f"- 总文件大小: {df['文件大小(KB)'].sum():.2f} KB")
    
    return df

# 批量提取示例
summary_df = batch_extract_and_summarize(test_dir)

print("\\n💡 实际应用：")
print("可以将汇总数据导出：")
print("summary_df.to_excel('word_summary_report.xlsx', index=False)")

---
# 第三部分：使用模板批量生成文档

## ⭐ Python独特优势：模板化自动生成

### docxtpl的核心优势：
- 使用Jinja2模板语法（支持变量、循环、条件）
- 在Word模板中直接标记变量位置
- 批量生成复杂格式的文档

### 应用场景：
- 批量生成合同（包含条件条款）
- 生成复杂格式的报告
- 根据数据动态生成表格

### ⭐ 重点复习：创建和使用Word模板

**模板语法：**
- `{{ variable }}` - 插入变量
- `{% for item in list %}...{% endfor %}` - 循环
- `{% if condition %}...{% endif %}` - 条件判断

**工作流程：**
1. 在Word中创建模板文档，插入占位符
2. 使用Python读取模板并填充数据
3. 批量生成最终文档

In [ ]:
# 创建Word模板文档
def create_template_document():
    """
    创建一个Word模板文档（手动创建的简化版本）
    
    在实际应用中，你会：
    1. 在Word中创建模板
    2. 使用 {{ variable }} 标记变量位置
    3. 保存为 .docx 文件
    """
    doc = Document()
    
    # 添加模板内容（包含占位符）
    doc.add_heading('销售合同', 0)
    doc.add_paragraph('合同编号：{{ contract_no }}')
    doc.add_paragraph('签订日期：{{ date }}')
    doc.add_paragraph()
    
    doc.add_heading('甲方信息', 2)
    doc.add_paragraph('公司名称：{{ company_name }}')
    doc.add_paragraph('联系人：{{ contact_person }}')
    doc.add_paragraph()
    
    doc.add_heading('合同金额', 2)
    doc.add_paragraph('总金额：{{ total_amount }}')
    doc.add_paragraph('付款方式：{{ payment_method }}')
    doc.add_paragraph()
    
    doc.add_heading('产品清单', 2)
    doc.add_paragraph('{% for item in items %}')
    doc.add_paragraph('- {{ item.name }}: {{ item.quantity }} 件，单价 {{ item.price }}')
    doc.add_paragraph('{% endfor %}')
    
    # 保存模板
    template_path = test_dir / "contract_template.docx"
    doc.save(template_path)
    print(f"✓ 模板已创建: {template_path}")
    return template_path

# 创建模板
template_path = create_template_document()

print("\\n💡 实际使用时：")
print("1. 在Word中打开模板文档")
print("2. 直接编辑并插入 {{ 变量名 }} 占位符")
print("3. 保存模板文件")

In [ ]:
# 使用模板批量生成文档
def generate_from_template(template_path, data, output_path):
    """
    使用模板生成Word文档
    
    参数：
        template_path: 模板文件路径
        data: 要填充的数据（字典）
        output_path: 输出文件路径
    """
    # 加载模板
    doc = DocxTemplate(template_path)
    
    # 渲染模板（填充数据）
    doc.render(data)
    
    # 保存文档
    doc.save(output_path)
    print(f"✓ 已生成: {output_path}")

# 准备数据
contract_data = {
    'contract_no': 'CT-2024-001',
    'date': '2024-01-15',
    'company_name': '某某科技有限公司',
    'contact_person': '张经理',
    'total_amount': '¥150,000',
    'payment_method': '分期付款',
    'items': [
        {'name': '产品A', 'quantity': 100, 'price': '¥800'},
        {'name': '产品B', 'quantity': 50, 'price': '¥1,400'},
    ]
}

# 生成文档
try:
    generate_from_template(
        template_path,
        contract_data,
        test_dir / "generated_contract.docx"
    )
except Exception as e:
    print(f"注意：使用docxtpl需要在Word中创建的模板，这里的示例模板可能不兼容")
    print(f"实际应用中，请在Word中创建带有 {{{{ variable }}}} 占位符的模板")

print("\\n💡 批量生成示例：")
print("# 从数据库或Excel读取数据")
print("contracts = pd.read_excel('contracts_data.xlsx')")
print("for i, contract in contracts.iterrows():")
print("    generate_from_template(template_path, contract.to_dict(), f'contract_{i}.docx')")

---
# 第四部分：批量修改和格式化文档

## ⭐ Python独特优势：批量格式统一

### 应用场景：
- 批量统一所有文档的字体、字号
- 批量添加页眉、页脚
- 批量替换文本内容
- 批量调整表格样式

### ⭐ 重点复习：批量修改文档格式

**Python的优势：**
- 一次性修改成百上千个文档
- 确保所有文档格式完全一致
- 可以设置复杂的格式规则

**实际应用：**
- 统一公司所有文档的格式标准
- 批量调整文档样式
- 批量添加水印、页码

In [ ]:
# 批量修改文档格式
def batch_format_documents(input_dir, output_dir, format_settings):
    """
    批量修改Word文档的格式
    
    参数：
        input_dir: 输入目录
        output_dir: 输出目录
        format_settings: 格式设置字典
    """
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)
    
    docx_files = list(input_dir.glob("*.docx"))
    print(f"找到 {len(docx_files)} 个文档，开始批量格式化...\\n")
    
    for docx_file in docx_files:
        try:
            doc = Document(docx_file)
            
            # 修改所有段落的格式
            for para in doc.paragraphs:
                if para.text.strip():  # 忽略空段落
                    # 设置字体
                    for run in para.runs:
                        run.font.name = format_settings.get('font_name', '宋体')
                        run.font.size = Pt(format_settings.get('font_size', 12))
            
            # 保存修改后的文档
            output_path = output_dir / docx_file.name
            doc.save(output_path)
            print(f"✓ {docx_file.name}")
        
        except Exception as e:
            print(f"✗ {docx_file.name}: {e}")
    
    print(f"\\n✓ 批量格式化完成！")
    print(f"输出目录: {output_dir.absolute()}")

# 格式设置
format_settings = {
    'font_name': '微软雅黑',
    'font_size': 11,
}

# 批量格式化
batch_format_documents(
    test_dir,
    test_dir / "formatted_docs",
    format_settings
)

print("\\n💡 更多格式设置：")
print("- 字体颜色：run.font.color.rgb = RGBColor(0, 0, 255)")
print("- 加粗：run.font.bold = True")
print("- 斜体：run.font.italic = True")
print("- 段落对齐：para.alignment = WD_ALIGN_PARAGRAPH.CENTER")

---
# 总结：Python处理Word的独特优势

## 🎯 核心优势总结

### 1. 批量自动化
- ✅ 批量创建/修改成百上千个文档
- ✅ 完全自动化，无需人工干预
- ✅ 可以定时自动执行
- ❌ Word软件：只能一个一个手动处理

### 2. 数据提取与分析
- ✅ 批量提取文本、表格数据
- ✅ 转换为DataFrame进行分析
- ✅ 汇总多个文档的数据
- ❌ Word软件：只能手动复制粘贴

### 3. 模板化批量生成
- ✅ 根据数据批量生成个性化文档
- ✅ 支持复杂的条件和循环逻辑
- ✅ 确保所有文档格式一致
- ❌ Word软件：邮件合并功能有限

### 4. 数据源集成
- ✅ 从数据库、Excel、API读取数据
- ✅ 与业务系统集成
- ✅ 自动化工作流
- ❌ Word软件：数据源集成困难

## 📚 常用功能速查

| 需求 | 核心函数/方法 | 说明 |
|-----|--------------|------|
| 创建文档 | `Document()` | 创建新文档 |
| 打开文档 | `Document(path)` | 打开已有文档 |
| 添加标题 | `doc.add_heading()` | 添加标题段落 |
| 添加段落 | `doc.add_paragraph()` | 添加普通段落 |
| 添加表格 | `doc.add_table()` | 添加表格 |
| 提取文本 | `para.text` | 获取段落文本 |
| 提取表格 | `doc.tables` | 获取所有表格 |
| 使用模板 | `DocxTemplate()` | 模板引擎 |

## 🚀 实际应用建议

1. **从小做起** - 先自动化一个简单任务
2. **模板优先** - 复杂文档使用模板更高效
3. **数据驱动** - 将数据和格式分离
4. **批量处理** - 充分利用Python的批量能力
5. **错误处理** - 添加异常处理确保稳定性

## 💡 下一步学习

- 结合pandas处理更复杂的数据
- 学习更多docxtpl模板语法
- 创建Web应用提供文档生成服务
- 与数据库集成实现完整工作流

---

**记住：Python处理Word的核心价值是批量化、自动化、智能化！**

## 清理测试文件

In [ ]:
# 如需清理测试文件，取消下面代码的注释
"""
import shutil

if test_dir.exists():
    shutil.rmtree(test_dir)
    print(f"已清理测试文件夹: {test_dir}")
"""